http://studyai.site/2017/02/26/%E3%80%90Tensorflow%20r1.0%20%E6%96%87%E6%A1%A3%E7%BF%BB%E8%AF%91%E3%80%91%E6%B7%B1%E5%85%A5MNIST--%E4%B8%93%E5%AE%B6%E7%BA%A7/

In [14]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
sess = tf.InteractiveSession() # 不同session之间数据计算独立

In [7]:
# 权重
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)# 初始权重，加入标准差为0.1的随机噪声
    return tf.Variable(initial)

# 偏置
def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [16]:
# 卷积层
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') # strides:卷积核参数；[0:1]卷积核大小；[2]通道；[3]卷积核个数

# MP层
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') # [batch, height, width, channels]

In [17]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image = tf.reshape(x, [-1,28,28,1]) # 28x28 images with channel 1

In [18]:
# layer-1 conv
W_conv1 = weight_variable([5,5,1,32]) # 5x5 kernel 32 counts
b_conv1 = bias_variable([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [23]:
h_pool1

<tf.Tensor 'MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [24]:
# layer-2 conv
W_conv2 = weight_variable([5,5,32,64]) # 5x5 kernel 64 counts (32, since h_pool1 shape is 14x14x32)
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [26]:
h_pool2

<tf.Tensor 'MaxPool_2:0' shape=(?, 7, 7, 64) dtype=float32>

In [27]:
# 将tensor变形为一维向量
W_fc1 = weight_variable([7 * 7 * 64, 1024]) # 全连接层，1024神经元
b_fc1 = bias_variable([1024])
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [28]:
# Dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [30]:
# Softmax
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])
y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [31]:
# loss function
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

In [36]:
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))# define acc function

In [35]:
# train
sess.run(tf.global_variables_initializer())
for i in range(10):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
    train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})
    
# accuracy    
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

step 0, training accuracy 0.1
step 1, training accuracy 0.1
step 2, training accuracy 0.1
step 3, training accuracy 0.1
step 4, training accuracy 0.1
step 5, training accuracy 0.1
step 6, training accuracy 0.1
step 7, training accuracy 0.1
step 8, training accuracy 0.1
step 9, training accuracy 0.1
test accuracy 0.2455
